https://adventofcode.com/2019/day/15

In [1]:
import networkx as nx
from intcode_computer import Intcode

In [2]:
with open('data/15.txt') as fh:
    data = fh.read().strip()

In [131]:
class Droid:
    def __init__(self, data=data):
        self.ic = Intcode([int(x) for x in data.split(',')])
    
    def pos2move(self, oldpos, newpos):
       
        if oldpos == newpos:
            return
        
        x1, y1 = oldpos
        x2, y2 = newpos
        
        if abs(x2 - x1) + abs(y2 - y1) > 1:
            raise TooFarError(f"Move not possible: {oldpos} to {newpos}")
        
        if y2 - y1 == 1:
            m = 1
        elif y1 - y2 == 1:
            m = 2
        elif x2 - x1 == 1:
            m = 3
        elif x1 - x2 == 1:
            m = 4
        else:
            raise IndexError("pos2move can't happen")
        return m    
        
        
    def move(self, oldpos, newpos):
        if oldpos == newpos:
            print(f"Staying at {oldpos}")
            return
        return self.ic.step(self.pos2move(oldpos, newpos))
     

class DroidError(Exception):
    pass

class TooFarError(Exception):
    pass

## DFS

In [91]:
def listnabes(pos):
    x, y = pos
    return [(x, y+1), (x+1, y), (x, y-1), (x-1, y)]

In [193]:
dd = Droid()
frontier = [(0,0)]
visited = {(0,0): 1}
G = nx.Graph()
G.add_node((0,0), r=1)

oldpos = (0, 0)
while frontier:
    pos = frontier.pop()
    try:
        dd.move(oldpos, pos)
    except TooFarError:
#         print("Too far, looking for shortest path now")
        a = oldpos
        b = pos
        sp = nx.shortest_path(G, oldpos, pos)
#         print("shortest path", sp)
        for n in sp[1:]:
#             print(n)
            dd.move(a, n)
            a = n
    for nabe in listnabes(pos):
        if nabe in visited:
            continue
        r = dd.move(pos, nabe)
        if r is None:
            print("droid's ic says 99")
            break
        visited[nabe] = r
        if r == 2:
            O2 = nabe
        if r != 0:
            G.add_node(nabe, r=r)
            G.add_edge(pos, nabe)
            dd.move(nabe, pos)
            frontier.append(nabe)
    oldpos = pos
    if len(visited) % 100 == 0:
        print(len(visited))

Staying at (0, 0)
200
400
600
600
700
900
1100
1400
1500


In [194]:
printmaze(visited)

#...#...........#...#.............#.....
 ##.#.###.#####.#.#.#######.#####.#####.
#...#.#.#.#.......#.........#.....#...#.
#.###.#.#.###.###############.#####.#.#.
#.....#.#...#...#.......#.#.........#...
#.#####.###.###.#.#####.#.#.#######.####
#.....#...#.#.#.#.#.....#.#.#.....#.#...
 ####.#.#.#.#.#.#.#.#####.#.#.###.###.#.
#.......#.#.#...#.#.#.....#.#.#.#.....#.
#.#########.#####.#.#.#.###.#.#.#######.
#.#.......#.#.....#.#.#.....#.#.#...#...
#.#.#####.#.#.#####.#####.###.#.#.#.#.#.
#.#.#...#.#.......#.....#...#...#.#...#.
#.#.#.#.#.#######.#####.#######.#.######
#.#.#.#.#...#...#.....#.......#.#...#...
#.#.###.###.#.#.#######.###.###.###.#.#.
#.#.....#.#...#...#...#...#...#...#...#.
 ######.#.#### ##.#.#.#####.#.###.#####.
#.....#.#.....#.#...#.#...#.#.#...#.....
#.###.#.#.###.#.#####.#.#.###.#.###.###.
#.#.#...#.#.#.#.#...#.#.#.....#...#.#...
#.#.#####.#.#.#.#.###.#.#########.#.#.##
#.....#...#.#...#.....#.#.........#.#...
 ####.#.###.###.#######.#.#########.###.
#.....#.#.....#.

In [195]:
len(visited)

1659

In [196]:
O2

(-16, -12)

In [197]:
len(nx.shortest_path(G, (0,0), O2)) - 1

228

In [198]:
len(G)

799

In [109]:
def printmaze(D):
    points = list(D.keys())
    ymax = max(y for (x,y) in points)
    ymin = min(y for (x, y) in points)
    xmax = max(x for (x,y) in points)
    xmin = min(x for (x, y) in points)
    
    for y in reversed(range(ymin, ymax)):
        for x in range(xmin, xmax):
            v = D.get((x, y))
            if v is None:
                pt = ' '
            elif v == 0:
                pt = '#'
            elif v == 1:
                pt = '.'
            elif v == 2:
                pt = '@'
            else:
                raise IndexError("Bad point val")
            print(pt, end='')
        print()

## Part 2

In [199]:
list(nx.neighbors(G, O2))

[(-16, -13)]

In [205]:
list(nx.neighbors(G, (-16,-13)))

[(-16, -14), (-16, -12)]

In [206]:
breathable = {O2}
i = 0
spaces = len(G)
while len(breathable) < spaces:
    for node in list(breathable):
        for nabe in nx.neighbors(G, node):
            breathable.add(nabe)
    i += 1

In [207]:
i

348